# Mock

In [ ]:
import random

class Riddler:

    tries = ['попыток', 'попытка', 'попытки']

    def __init__(self):
        self.riddles = {'Маленький, серенький, на слона похож.': 'слоненок', 
                        'Над нами кверху ногами.': 'таракан', 
                        'Cиний, большой, с усами и полностью набит зайцами.': 'троллейбус'}

    def add_riddle(self, riddle: str, answer: str):
        """ Добавляет загадку в словарь """
        if not isinstance(riddle, str) or not isinstance(answer, str):
            print('Wrong type!!')
            return
        self.riddles[riddle] = answer

    def riddle(self):
        """ Печатает текст загадки и проверяет правильность ответов """
        question = random.choice(list(self.riddles.keys()))
        print('Загадка: ' + question)
        print('У вас 3 попытки!')
        for i in range(3,0, -1):
            answer = input()
            if answer == self.riddles[question]:
                print('Правильно!!!')
                return True
            print(f'У вас {i-1} {self.tries[i-1]}!')
        print('Правильный ответ: ' + self.riddles[question])
        return False
            

In [ ]:
riddler = Riddler()

In [ ]:
riddler.riddle()

In [ ]:
from unittest.mock import call

In [ ]:
import unittest

class RiddlerTestCase(unittest.TestCase):

    def setUp(self):
        self.riddler = Riddler()

    # тестируем метод add_riddle, все довольно просто
    def test_add_riddle_success(self):
        self.riddler.add_riddle('test', 'test')
        self.assertEqual(self.riddler.riddles['test'], 'test')

    def test_add_riddle_wrong_type(self):
        riddles_before = self.riddler.riddles.copy()
        self.riddler.add_riddle(123, 123)
        self.assertEqual(self.riddler.riddles, riddles_before)

    # как тестировать метод с пользовательским вводом? принтом? и рандомом?
    def test_riddle(self):
        pass



if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter


## unittest.mock.Mock

Mock - специальный объект на любой вызов, обращение к атрибуту или методу возвращающий новый объект Mock или то, что мы сами попросим. 

In [ ]:
from unittest.mock import Mock

In [ ]:
m = Mock()

In [ ]:
# вызов
m()

In [ ]:
# атрибут
m.attr

In [ ]:
# метод
m.method()

Можно задавать свои атрибуты и их значения

In [ ]:
m = Mock(my_attr=28)

In [ ]:
m.my_attr

### return_value

В параметр return_value передаем то, что хотим получить в результате вызова мок-объекта

In [ ]:
m = Mock(return_value=28)

In [ ]:
m()

### side_effect

В параметр side_effect можно передать много чего: 
 + любой итерируемый объект (тогда мок при каждом вызове будет возвращать следующий элемент итератора)
 + функцию, которая будет вызвана с переданными в исходную функцию парамерами вместо нее
 + или исключение (тогда оно будет поднято в процессе выполнения теста)       
 
Подробнее в документации https://docs.python.org/3/library/unittest.mock.html#unittest.mock.Mock.side_effect


Итерируемый объект:

In [ ]:
m = Mock(side_effect=[1,2,3,4,5])

In [ ]:
m()

In [ ]:
m()

In [ ]:
m()

In [ ]:
m = Mock(side_effect=[1,2,3,4,5])
for i in range(5):
    print(m())

Функция:

In [ ]:
def side_effect_callable(arg):
    values = {'a': 1, 'b': 2, 'c': 3}
    if arg in values:
        return values[arg]
    return 0

In [ ]:
m = Mock(side_effect=side_effect_callable)

In [ ]:
m('a')

In [ ]:
m('c')

In [ ]:
m('d')

### Проверка списка вызовов

Можно проверять сколько раз и с какими аргументами был бызван мок-объект:

In [ ]:
m.call_args_list # список вызовов

In [ ]:
m.assert_called() # был когда-либо вызван

In [ ]:
m.assert_called_once() # вызван ровно 1 раз

In [ ]:
m.assert_called_with('d') # проверяет аргументы последнего вызова

In [ ]:
from unittest.mock import call
# проверить, что в списке вызовов есть все нужные вызовы в заданном порядке
m.assert_has_calls([call('a'), call('c'), call('d')]) 

## подмена объектов с помощью patch

+ заменить один объект дургим на время тестов можно с помощью функции ***patch***
+ используется внутри менеджера контекстов
```
with patch('module.object.method', ...):
    ...
```
+ или в виде декоратора - аргументы будут те же
```
@patch('module.object.method', ...)
def test_something(...):
    ...
```
+ первый аргумент - путь до объекта/метода который надо заменить (через точки, так же как мы импортируем объекты)
+ следующие аргументы определяют на что и как именно заменить
+ подробнее про аргументы: https://docs.python.org/3/library/unittest.mock.html#patch   


**ВАЖНО - где именно заменять объект**
+ То есть какой именно путь писать в ***patch*** первым аргументом?
+ Основное правило - заменять объект нужно **там где он используется**, а не там откуда его импортировали. 
+ То есть если в модуле (***my_beautiful_module.py***), который мы хотим протестировать импортируется какой-то объект который мы хотим заменить на мок (в данном случае функция ***some_fucntion***)
```
from some_module import some_function
def my_function():
    result = some_method() + 1
    return result
```
+ То в тестах нужно делать вот так, (а не *'some_module.some_function'*)
```
@patch('my_beautiful_module.some_function', ...)
def test_my_function(...):
    ...
```
+ Обычно все работает, даже если делать неправильно, но далеко не всегда.
+ Подробнее про это: https://docs.python.org/3/library/unittest.mock.html#id6


In [ ]:
def greet_user():
    name = input('Представьтесь, пожалуйста')
    return('Привет, %s!' % name)

In [ ]:
import unittest
from unittest.mock import Mock, patch

class GreetUserTestCase(unittest.TestCase):
    
    @patch('builtins.input', Mock(return_value='Юрий'))
    def test_greet_user(self):
        self.assertEqual(greet_user(), 'Привет, Юрий!')
    
#     # эквивалентно
#     def test_greet_user(self):
#         with patch('builtins.input', Mock(return_value='Юрий')) as mock_input:
#             self.assertEqual(greet_user(), 'Привет, Юрий!')
        
    
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter

**Задание**: 
+ переписать тесты для метода greet_user, при условии замены ***return*** на ***print***
+ обязательно проверить, что именно выводится на экран!

In [ ]:
def greet_user():
    name = input('Представьтесь, пожалуйста')
    print('Привет, %s!' % name)

In [ ]:
import unittest
from unittest.mock import Mock, patch

class GreetUserTestCase(unittest.TestCase):
        
    def test_greet_user(self):
        pass
        
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter    

**Задание**:
+ переварить все рассказанное и показанное выше и написать тесты для метода ***riddle***
+ проверить нужно все варианты (угадывание с n-ой попытки, неугадывание) и не только возвращаемое значение, но и побочные эффекты (что печатается)
+ добиться 100% покрытия кода тестами